문제 1: 채널별 고객 획득 비용(CAC) 및 생애 가치(LTV) 분석
목표: 마케팅 채널의 진정한 ROI를 평가하기 위해 CAC와 LTV를 계산하고 최적 투자 전략을 제안하세요.
1. 데이터 로드 및 전처리
2. 채널별 CAC 계산
    - 2024년 데이터만 사용 - (2024년 마케팅데이터만 사용 -> `24년 채녈별 마케팅비용/24년 신규 획득 고객 수`)
    - 각 채널별 총 마케팅 비용 집계
    - 각 채널별 신규 고객 수 집계 (2024년 첫 거래 기준)
    - CAC = 총 마케팅 비용 / 신규 고객 수
    - organic과 referral 채널은 CAC = 0으로 처리
3. 고객별 LTV 계산 - 전체 구매 이력 기반
4. 채널별 LTV 및 ROI 분석 - (`24년 신규 고객들의 평균 LTV / 24년 CAC`)
    - 채널별 평균 LTV 계산
    - ROI = LTV / CAC 계산 (organic, referral은 무한대 처리)
    - payback period = CAC / (월평균 구매금액) 계산
5. 시각화 및 전략 제안 (15분)
    - CAC vs LTV 산점도 차트
    - 채널별 ROI 막대 차트
    - 마케팅 예산 재배분 제안 (현재 vs 최적 배분)

In [33]:
# 1. 데이터 로드 및 전처리
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

cs = pd.read_csv('./customer_satisfaction.csv', encoding='UTF-8')
trasactions = pd.read_csv('./customer_transactions.csv', encoding='UTF-8')
marketing = pd.read_csv('./marketing_performance.csv', encoding='UTF-8')

In [38]:
# 2. 채널별 CAC 계산
# - 2024년 데이터만 사용
# - 각 채널별 총 마케팅 비용 집계
# - 각 채널별 신규 고객 수 집계 (2024년 첫 거래 기준)
# - CAC = 총 마케팅 비용 / 신규 고객 수
# - organic과 referral 채널은 CAC = 0으로 처리

# 코드 실행 상태가 초기화되어, 다시 모든 파일 불러오기부터 시작

# 날짜형 컬럼 변환
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'], errors='coerce')
marketing['month'] = pd.to_datetime(marketing['month'], errors='coerce')

# 2024년 데이터 필터링
transactions_2024 = transactions[transactions['transaction_date'].dt.year == 2024]
marketing_2024 = marketing[marketing['month'].dt.year == 2024]

# 신규 고객 판별 (2024년에 첫 거래한 고객)
first_purchase = transactions.groupby('customer_id')['transaction_date'].min().reset_index()
first_purchase_2024 = first_purchase[first_purchase['transaction_date'].dt.year == 2024]
new_customers_2024 = pd.merge(first_purchase_2024, transactions[['customer_id', 'acquisition_channel']], on='customer_id', how='left').drop_duplicates('customer_id')

# 채널별 신규 고객 수
new_customer_counts = new_customers_2024['acquisition_channel'].value_counts().rename_axis('channel').reset_index(name='new_customers')

# 채널별 총 마케팅 비용
marketing_costs = marketing_2024.groupby('channel')['marketing_spend'].sum().reset_index()

# CAC 계산 (organic, referral = 0 처리)
cac = pd.merge(marketing_costs, new_customer_counts, on='channel', how='outer').fillna(0)
cac['CAC'] = cac.apply(lambda x: 0 if x['channel'] in ['organic', 'referral'] else (x['marketing_spend'] / x['new_customers']) if x['new_customers'] > 0 else None, axis=1)

transactions_2024

,customer_id,registration_date,acquisition_channel,customer_segment,transaction_date,order_value,product_category
1,1,2023-04-13,email,premium,2024-01-15,141.21,Electronics
2,1,2023-04-13,email,premium,2024-02-21,54.80,Electronics
7,1,2023-04-13,email,premium,2024-02-14,133.51,Home
11,2,2023-01-21,paid_search,premium,2024-02-13,134.87,Fashion
12,2,2023-01-21,paid_search,premium,2024-02-15,132.21,Electronics
...,...,...,...,...,...,...,...
4555,996,2023-12-29,organic,basic,2024-07-18,44.13,Fashion
4556,997,2023-04-26,social_media,basic,2024-04-12,40.51,Electronics
4561,999,2023-06-04,social_media,basic,2024-04-15,31.90,Books
4562,999,2023-06-04,social_media,basic,2024-02-11,29.22,Home


In [ ]:
# 3. 고객별 LTV 계산


In [ ]:
# 4. 채널별 LTV 및 ROI 분석
#     - 채널별 평균 LTV 계산
#     - ROI = LTV / CAC 계산 (organic, referral은 무한대 처리)
#     - payback period = CAC / (월평균 구매금액) 계산


In [ ]:
# 5. 시각화 및 전략 제안 (15분)
#     - CAC vs LTV 산점도 차트
#     - 채널별 ROI 막대 차트
#     - 마케팅 예산 재배분 제안 (현재 vs 최적 배분)

문제 2: 고객 코호트 분석 및 유지율 개선 전략
목표: "월별 가입 코호트"의 유지율과 매출 기여도를 분석하여 고객 유지 전략을 수립하세요.
1. 코호트 그룹 생성
2. 월별 유지율 계산
    - 각 거래의 코호트 대비 경과 개월 수 계산
    - 코호트별 Month 0, 1, 2, ... 12에서의 활성 고객 수
    - 유지율 테이블 생성 (코호트 × 경과월)
    - 평균 유지율 곡선 계산
3. 코호트별 매출 기여도 분석
    - 각 코호트의 월별 누적 매출 계산
    - 코호트별 고객 1인당 누적 매출 (ARPU) 계산
    - 6개월 후, 12개월 후 예상 LTV 추정
4. 유지율 히트맵 시각화
5. 개선 전략 수립
    - 유지율이 급격히 감소하는 시점 식별
    - 고성과 코호트 vs 저성과 코호트 특성 비교
    - 유지율 개선을 위한 액션 아이템 3가지 제안

In [ ]:
# 1. 코호트 그룹 생성

In [ ]:
# 2. 월별 유지율 계산
# - 각 거래의 코호트 대비 경과 개월 수 계산
# - 코호트별 Month 0, 1, 2, ... 12에서의 활성 고객 수
# - 유지율 테이블 생성 (코호트 × 경과월)
# - 평균 유지율 곡선 계산

In [ ]:
# 3. 코호트별 매출 기여도 분석
#     - 각 코호트의 월별 누적 매출 계산
#     - 코호트별 고객 1인당 누적 매출 (ARPU) 계산
#     - 6개월 후, 12개월 후 예상 LTV 추정

In [ ]:
# 4. 유지율 히트맵 시각화


In [ ]:
# 5. 개선 전략 수립
#     - 유지율이 급격히 감소하는 시점 식별
#     - 고성과 코호트 vs 저성과 코호트 특성 비교
#     - 유지율 개선을 위한 액션 아이템 3가지 제안